# Sold houses for 78613

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import time
import numpy as np


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
def getAndParseURL(url):
    result = requests.get(url,headers=headers, timeout=5)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)


base="https://www.zillow.com"
main_url="https://www.zillow.com/homes/recently_sold/Leander-TX-78641/house_type/92576_rid/36m_days/2001-2001_built/"


zipcode=[78613]
year=np.arange(2000,2019)


#all searches
allsearchurl=[]
for i in zipcode:
    for j in year:
            url1="https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-"
            url2="/house_type/92551_rid/36m_days/"
            years="{}-{}_built/".format(j,j)
            allsearchurl.append(url1+str(i)+url2+years)
allsearchurl

['https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2000-2000_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2001-2001_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2002-2002_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2003-2003_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2004-2004_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2005-2005_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2006-2006_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36m_days/2007-2007_built/',
 'https://www.zillow.com/homes/recently_sold/Cedar-Park-TX-78613/house_type/92551_rid/36

In [3]:
allsearchpages = []
for i in allsearchurl:

    delays = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    delay = np.random.choice(delays)
    time.sleep(delay)

    allapages = [i]
    index = 1
    a = getAndParseURL(i)
    while a.find("li", {"class": "zsg-pagination-next"}):
        nextpage = base + a.find("li", {"class": "zsg-pagination-next"}).a["href"]
        allapages.append(nextpage)
        a = getAndParseURL(nextpage)

    allsearchpages.extend(allapages)


allsearchpagelinks=[]
for k in allsearchpages:
    delays = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    delay = np.random.choice(delays)
    time.sleep(delay)

    a=getAndParseURL(k)
    allsearchpagelinks.extend(base+i["href"] for i in a.find("div",{"id":"list-results"}).findAll("a",{"class":"zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link"}))





beds = []
baths = []
sqfts = []
solddates = []
soldprices = []
estimateprices = []
housetypes = []
YearBuilts = []
Heatings = []
Coolings = []
parkings = []
floors = []
lotsizes = []
names = []
numberOfRoomss = []
streetAddresss = []
addressLocalitys = []
addressRegions = []
postalCodes = []
latitudes = []
longitudes = []
houseurls = []

for k in allsearchpagelinks:
    delays = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    delay = np.random.choice(delays)
    time.sleep(delay)
    a = getAndParseURL(k)
    try:
        if a.find("span", {"class": "ds-status-details"}).text == 'House for rent':
            continue
    except:

        try:
            info = a.find("h3", {"class": "edit-facts-light"}).text
        except:
            a = getAndParseURL(k)
            info = a.find("h3", {"class": "edit-facts-light"}).text
        try:
            # beds
            bed = int(("").join((info.split("\xa0")[1]).split(" beds")))
            beds.append(bed)
        except:
            beds.append(None)

        try:
            # bath
            bath = float(("").join((info.split("\xa0")[2]).split(" baths")))
            baths.append(bath)
        except:
            baths.append(None)

        try:
            # sqrt
            sqrt = info.split("\xa0")[3]
            sqft = int(("").join(re.split(" sqft|,", sqrt)))
            sqfts.append(sqft)
        except:
            sqfts.append(None)

        try:
            #solddate
            solddate = a.find("div", {"class": "zsg-fineprint date-sold"}).text
            solddate = ("").join(re.split(" ", solddate)[-1])
            solddates.append(solddate)
        except:
            solddates.append(None)

        try:
            # soldprice
            soldprice = a.find("div", {"class": "status"}).text
            soldprice = int(("").join(re.split(" |,|\$|Sold:", soldprice)))
            soldprices.append(soldprice)
        except:
            soldprices.append(None)

        try:
            # estimateprice
            estimateprice = a.find("div", {"class": "zestimate primary-quote"}).text
            estimateprice = int(("").join(re.split(",|Zestimate®: \$", estimateprice)))
            estimateprices.append(estimateprice)
        except:
            estimateprices.append(None)

        factlabel = [((i.text).replace('\xa0', '')).replace(':', '') for i in
                     a.find("div", {"class": "home-facts-at-a-glance-section"}).findAll("div", {"class": "fact-label"})]
        factvalue = [i.text for i in
                     a.find("div", {"class": "home-facts-at-a-glance-section"}).findAll("div", {"class": "fact-value"})]
        factdict = dict(zip(factlabel, factvalue))

        # housetype
        housetype = factdict['Type']
        housetypes.append(housetype)

        # 'Year Built'
        YearBuilt = int(factdict['Year Built'])
        YearBuilts.append(YearBuilt)

        # Heating
        Heating = factdict['Heating']
        Heatings.append(Heating)

        # 'Cooling'
        Cooling = factdict['Cooling']
        Coolings.append(Cooling)

        # parking
        parking = (("").join(re.split(" spaces|space", factdict['Parking'])))
        parkings.append(parking)

        try:
            # floor
            t1 = a.findAll("div", {"class": "home-details-facts-category-group-container"})
            t2 = [i for i in t1 if i.findAll(text=re.compile('Construction'))]
            flr = int(t2[0].findAll("div", {"class": "fact-value"})[-1].text)
            floors.append(flr)
        except:
            floors.append(None)

        # lotsize
        try:
            lotsize = int(("").join(re.split(" sqft|,|\.| acres", factdict['Lot'])))
            lotsizes.append(lotsize)
        except:
            lotsizes.append(None)

        s = a.find("div", {"class": "home-details-summary-and-price"}).find("script",
                                                                            {"type": "application/ld+json"}).text

        name = json.loads(s)['name']  # 1414 Ridgewood Dr, Leander, TX 78641
        names.append(name)

        numberOfRooms = json.loads(s)['numberOfRooms']  # 3
        numberOfRoomss.append(numberOfRooms)

        streetAddress = json.loads(s)['address']["streetAddress"]  # 1414 Ridgewood Dr
        streetAddresss.append(streetAddress)

        addressLocality = json.loads(s)['address']['addressLocality']  # Leander
        addressLocalitys.append(addressLocality)

        addressRegion = json.loads(s)['address']['addressRegion']  # TX
        addressRegions.append(addressRegion)

        postalCode = json.loads(s)['address']['postalCode']  # '78641'
        postalCodes.append(postalCode)

        latitude = json.loads(s)["geo"]['latitude']
        latitudes.append(latitude)

        longitude = json.loads(s)["geo"]['longitude']
        longitudes.append(longitude)

        houseurl = base + json.loads(s)["url"]
        houseurls.append(houseurl)





In [4]:
df = pd.DataFrame({"beds":beds,
                   "baths":baths,
                   "sqft":sqfts,
                   "solddate":solddates,
                   "soldprice":soldprices,
                    "estimateprice":estimateprices,
                    "housetype":housetypes,
                    "YearBuilt":YearBuilts,
                    "Heating":Heatings,
                    "Cooling":Coolings,
                    "parking":parkings,
                    "lotsize":lotsizes,
                    "name":names,
                    "numberOfRooms":numberOfRoomss,
                    "streetAddress":streetAddresss,
                    "addressLocality":addressLocalitys,
                    "addressRegion":addressRegions,
                    "postalCode":postalCodes,
                    "latitude":latitudes,
                    "longitude":longitudes,
                    "houseurl":houseurls

  })

df.to_csv('zillow-78613-sold.csv')

In [5]:
df

,beds,baths,sqft,solddate,soldprice,estimateprice,housetype,YearBuilt,Heating,Cooling,...,lotsize,name,numberOfRooms,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,houseurl
0,3.0,2.0,1478.0,04/22/19,224778.0,243589.0,Single Family,2000,Other,Central,...,5662.0,"702 Le Ann Ln, Cedar Park, TX 78613",3.0,702 Le Ann Ln,Cedar Park,TX,78613,30.518779,-97.817411,https://www.zillow.com/homedetails/702-Le-Ann-...
1,4.0,2.5,2665.0,04/12/19,250000.0,257730.0,Single Family,2000,Forced air,Central,...,8276.0,"2605 Byfield Dr, Cedar Park, TX 78613",4.0,2605 Byfield Dr,Cedar Park,TX,78613,30.540536,-97.845924,https://www.zillow.com/homedetails/2605-Byfiel...
2,3.0,3.0,2255.0,04/05/19,269000.0,270245.0,Single Family,2000,"Forced air, Other",Central,...,7666.0,"1622 Abbey Ln, Cedar Park, TX 78613",3.0,1622 Abbey Ln,Cedar Park,TX,78613,30.522559,-97.854897,https://www.zillow.com/homedetails/1622-Abbey-...
3,3.0,2.0,1820.0,03/22/19,252883.0,273292.0,Single Family,2000,Forced air,Central,...,8624.0,"1007 Cashew Ln, Cedar Park, TX 78613",3.0,1007 Cashew Ln,Cedar Park,TX,78613,30.507623,-97.804945,https://www.zillow.com/homedetails/1007-Cashew...
4,5.0,5.0,3608.0,03/01/19,450000.0,435462.0,Single Family,2000,Other,Central,...,8887.0,"1818 Cattle Dr, Cedar Park, TX 78613",5.0,1818 Cattle Dr,Cedar Park,TX,78613,30.487773,-97.846532,https://www.zillow.com/homedetails/1818-Cattle...
5,3.0,2.0,2054.0,02/19/19,325000.0,327848.0,Single Family,2000,Forced air,Central,...,8232.0,"106 Cluck Creek Trl, Cedar Park, TX 78613",3.0,106 Cluck Creek Trl,Cedar Park,TX,78613,30.506421,-97.830857,https://www.zillow.com/homedetails/106-Cluck-C...
6,5.0,3.0,3268.0,02/15/19,372248.0,356939.0,Single Family,2000,Forced air,Central,...,8842.0,"2110 Rachel Rdg, Cedar Park, TX 78613",5.0,2110 Rachel Rdg,Cedar Park,TX,78613,30.486937,-97.850789,https://www.zillow.com/homedetails/2110-Rachel...
7,3.0,2.0,1820.0,12/20/18,257852.0,284376.0,Single Family,2000,Forced air,Central,...,8712.0,"602 S Lynnwood Trl, Cedar Park, TX 78613",3.0,602 S Lynnwood Trl,Cedar Park,TX,78613,30.507676,-97.803343,https://www.zillow.com/homedetails/602-S-Lynnw...
8,4.0,2.0,1825.0,12/12/18,240263.0,253790.0,Single Family,2000,Forced air,Central,...,5445.0,"718 Le Ann Ln, Cedar Park, TX 78613",4.0,718 Le Ann Ln,Cedar Park,TX,78613,30.519679,-97.818170,https://www.zillow.com/homedetails/718-Le-Ann-...
9,3.0,2.5,2846.0,11/26/18,272942.0,249851.0,Single Family,2000,Forced air,Central,...,6577.0,"1037 Silverstone Ln, Cedar Park, TX 78613",3.0,1037 Silverstone Ln,Cedar Park,TX,78613,30.519607,-97.844708,https://www.zillow.com/homedetails/1037-Silver...
